# Creating and analysing the simulated datasets #
Current code includes: 
Number of transcripts: each transcript has its own SNPs, each with their own error rate, and its own mobility status.

Number of SNPs: the number of SNPs per transcript (currently fixed).

Number of replicates: We create 2 homograft files (Nhom1 and Nhom2), and then no_reps heterograft files. 

# Method A #

If a single SNP has a read depth >= min_read_thresh, then it is flagged as being mobile. If 1 or more SNPs are flagged as being mobile, then the transcript is flagged as being mobile.

# Handling SNPs #

Bayes Factors: sum them all (should we point out in the paper that this is a useful advantage of this method?)

Method A: snp_thresh SNPs need to have reads mapping to the other ecotype, in order for a mobile classification to be given for the transcript

Method B: snp_thres SNPs need to have reads mapping to the other ecotype, in order for a mobile classification to be given for the transcript

# Handling replicates #

Bayes Factors: Again, very simple. We sum across replicates

Method A: A transcript needs to have been given a mobile assignment in rep_thresh replicates, in order for a final mobile assignment to be given.

Method B: A transcript needs to have been given a mobile assignment in rep_thresh replicates, in order for a final mobiel assigment to be given.

## Figure 4

 Filtering SNPs based on observed errors or determining mobile transcripts based on absolute read counts leads to poor classification accuracy. Using our Bayesian approach we assign a transcript as being graft-mobile if $\log_{10} B_{21} \ge 1$. Methods A and B are explained in the main text. Here we have three replicates per transcript. The Bayesian approach sums up the evidence over replicates whereas Methods A and B require that two out of three replicates show evidence for mobility. Both plots shows the accuracy, (TP + TN) / (TP + TN + FP + FN), of each method over 1000 simulated datasets for different read depths, $N$, for an error rate of $q=0.01$. The left plot shows how the accuracy varies for different values of $N$ for homograft read depths equal to $N$. The right plot shows how the accuracy varies for different values of $N$ for fixed homograft read depths of 1000. The convergence of Method A and B towards $\approx$0.5 is a consequence of the balanced dataset with a 1:1 ratio of mobile:non-mobile transcripts, meaning that their performance is essentially little better than random. For an unbalanced dataset the accuracy could drop below 0.5 for Method A and B. 

In [ ]:
## Code to run the simulations for Figure 4
import baymobil.baymobil as baymob
import baymobil.simulations as sim
import baymobil.plot_data as plot_data

## Run the code in the folder for Figure 4
%cd   "/Users/tomkinsm/baymobil/Figure4/N/"
## Running and evaluating the simulations
## Each simulation is designed to investigate the accuracy of the results as a function of some parameter. The values for this parameter must be defined in the "parameters.cfg" file
parameter_func = "N"
## Create the datasets. This function creates no_reps .csv files, storing them in the "output" folder. This will delete any previous files, so make sure you have moved anything that you still need.
sim.create_simulated_data(parameter_func)
## Now, we load in and combine all of the replicate data, according to the above rules
df = plot_data.load_data()
## Plot and compare the accuracy (TP + TN) / (TP + TN + FP + FN) of the three different methods on our simulated datasets
plot_data.plot_data_all(df,parameter_func)

%cd   "/Users/tomkinsm/baymobil/Figure4/fixedNhom/"
parameter_func = "N"
sim.create_simulated_data(parameter_func)
df = plot_data.load_data()
plot_data.plot_data_all(df, parameter_func)


In [ ]:
## Code to run the simulations for Figure 5


import baymobil.baymobil as baymob
import baymobil.simulations as sim
import baymobil.plot_data as plot_data
import pandas as pd

%cd   "/Users/tomkinsm/baymobil/Figure5/"
sim.create_simulated_data("N")



In [ ]:
## Load in and process the data
## As there are two parameters being varied here (N and q), we can't use the built in plotting function

## Load in the data
df = pd.read_csv("SNP_wise_values.csv")
## Group by transcript
print(df)

df_transcript = df.groupby(["transcript","N","q"]).sum().reset_index()
print(df_transcript)

## Calculate TP, TN, FP, FN
df_transcript["TP"] = 0
df_transcript["TN"] = 0
df_transcript["FP"] = 0
df_transcript["FN"] = 0

df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.log10BF>=1),"TP"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.log10BF<1),"TN"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.log10BF>=1),"FP"] = 1
df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.log10BF<1),"FN"] = 1

print(df_transcript)

final_results = df_transcript.groupby(["N","q"]).sum()[["TP","TN","FP","FN"]].reset_index()

print(final_results)

In [ ]:
## Plot the pie-chart plots
## Create the figures separately and then add in the axes later

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

def plot_pie(x, ax):

    tp = float(x["TP"])
    tn = float(x["TN"]) 
    fp = float(x["FP"]) 
    fn = float(x["FN"]) 

    total = tp + tn + fp + fn
    
    tp = tp / total
    tn = tn / total
    fp = fp / total
    fn = fn / total

    n = float(x["N"])
    q = float(x["q"])

    ax.pie([tp,tn,fp,fn], center = ([n, q]), radius = 0.05, colors=['b','g','r','y'], normalize = True)

matplotlib.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(1,1, figsize=(8, 8), dpi=300)
fig.patch.set_facecolor('white')

## Bayes factors
df_transcript = df.groupby(["transcript","N","q"]).sum().reset_index()

## Calculate TP, TN, FP, FN
df_transcript["TP"] = 0
df_transcript["TN"] = 0
df_transcript["FP"] = 0
df_transcript["FN"] = 0

df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.log10BF>=1),"TP"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.log10BF<1),"TN"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.log10BF>=1),"FP"] = 1
df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.log10BF<1),"FN"] = 1

final_results = df_transcript.groupby(["N","q"]).sum()[["TP","TN","FP","FN"]].reset_index()
test = final_results.copy()
test["N"] = test["N"] / test["N"].max()
test["q"] = test["q"] / test["q"].max()
test.apply(lambda x: plot_pie(x,ax), axis=1)
x_ticks = np.arange(100,1000,100)
y_ticks = np.arange(0.001,0.01,0.001)
#plt.xticks(x_ticks)
#plt.yticks(y_ticks)
#plt.legend(['TP', 'TN','FP','FN'])
ax.axis('tight')

plt.savefig("bayes_pie.svg",dpi=300)
plt.show()


## Same plot for Method A
fig, ax = plt.subplots(1,1, figsize=(8, 8), dpi=300)
fig.patch.set_facecolor('white')

snp_thresh = 2
df_transcript = df.groupby(["transcript","N","q"]).sum().reset_index()
## Calculate TP, TN, FP, FN
df_transcript["TP"] = 0
df_transcript["TN"] = 0
df_transcript["FP"] = 0
df_transcript["FN"] = 0

df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.Method_A>=snp_thresh),"TP"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.Method_A<snp_thresh),"TN"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.Method_A>=snp_thresh),"FP"] = 1
df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.Method_A<snp_thresh),"FN"] = 1

final_results = df_transcript.groupby(["N","q"]).sum()[["TP","TN","FP","FN"]].reset_index()
test = final_results.copy()
test["N"] = test["N"] / test["N"].max()
test["q"] = test["q"] / test["q"].max()
test.apply(lambda x: plot_pie(x,ax), axis=1)
x_ticks = np.arange(100,1000,100)
y_ticks = np.arange(0.001,0.01,0.001)
#plt.xticks(x_ticks)
#plt.yticks(y_ticks)
#plt.legend(['TP', 'TN','FP','FN'])
#ax.set_title("Method A")
ax.axis('tight')
plt.savefig("methoda_pie.svg",dpi=300)
plt.show()

## Same plot for Method B
fig, ax = plt.subplots(1,1, figsize=(8, 8), dpi=300)
fig.patch.set_facecolor('white')
df_transcript = df.groupby(["transcript","N","q"]).sum().reset_index()

## Calculate TP, TN, FP, FN
df_transcript["TP"] = 0
df_transcript["TN"] = 0
df_transcript["FP"] = 0
df_transcript["FN"] = 0

df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.Method_B>=snp_thresh),"TP"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.Method_B<snp_thresh),"TN"] = 1
df_transcript.loc[(df_transcript.mobile == 0) & (df_transcript.Method_B>=snp_thresh),"FP"] = 1
df_transcript.loc[(df_transcript.mobile > 0) & (df_transcript.Method_B<snp_thresh),"FN"] = 1

final_results = df_transcript.groupby(["N","q"]).sum()[["TP","TN","FP","FN"]].reset_index()

test = final_results.copy()
test["N"] = test["N"] / test["N"].max()
test["q"] = test["q"] / test["q"].max()
test.apply(lambda x: plot_pie(x,ax), axis=1)
#plt.legend(['TP', 'TN','FP','FN'])
##ax.set_title("Method B")
ax.axis('tight')
plt.savefig("methodb_pie.svg",dpi=300)
plt.show()


In [ ]:
## Run a test of the Thieme data, to see if the new code gives the same results

import baymobil.baymobil as baymob
import baymobil.simulations as sim
import baymobil.plot_data as plot_data
import os
from os import listdir, mkdir, getcwd
from os.path import isfile, join, isdir
import pandas as pd

# Navigate to the correct folder
%cd   "/Users/tomkinsm/mRNA_analysis/"

## Thieme homfiles dictionary
def get_hom_data_thieme(filename):
    if filename.lower().startswith('col'):
        if "root" in filename:
            hom1 = "C-C-root-FN.csv"
            hom2 = "P-P-root-FN.csv"
        else:
            hom1 = "C-C-shoot-FN.csv"
            hom2 = "P-P-shoot-FN.csv"
    if filename.lower().startswith('ped'):
        if "root" in filename:
            hom2 = "C-C-root-FN.csv"
            hom1 = "P-P-root-FN.csv"
        else:
            hom2 = "C-C-shoot-FN.csv"
            hom1 = "P-P-shoot-FN.csv"
    if filename.startswith('FN'):
        ## FN files: col shoot, ped root
        if "root" in filename:
            hom2 = "C-C-root-FN.csv"
            hom1 = "P-P-root-FN.csv"
        else:
            hom1 = "C-C-shoot-FN.csv"
            hom2 = "P-P-shoot-FN.csv"
    return hom1, hom2

    ## Thieme / col-ped data
def format_hom_data_col_ped(hompath, thisfile):
    df = pd.read_csv(hompath + thisfile, delimiter="\t", low_memory = False)
    df = df[["SNP","depth","colDepth","lerDepth"]]
    ## Different handling based on file contents
    if thisfile.lower().startswith("c"):
        df.rename(columns={"colDepth":"eco1","lerDepth":"n", "depth":"N"}, inplace=True)
    elif thisfile.lower().startswith("p"):
        df.rename(columns={"lerDepth":"eco1","colDepth":"n", "depth":"N"}, inplace=True)
    else: 
        print("Error!")
    cols = df.columns.drop("SNP")
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    df.dropna(inplace=True)
    df = df[df["N"]>0]
    filename = thisfile.split(".")[0]
    ## We need a new folder for the output files: check if this exists
    clean_data_path = os.path.join(hompath, "clean_data/")
    if os.path.exists(clean_data_path) == False:
        os.makedirs(clean_data_path)
    df.to_csv(clean_data_path + filename + ".csv",index=None)

def format_het_data_col_ped(hetpath, thisfile):
    df = pd.read_csv(hetpath + thisfile, delimiter="\t", low_memory = False)
    df = df[["SNP","depth","colDepth","lerDepth"]]
    ## Different handling based on file contents
    if thisfile.lower().startswith("col"):
        df.rename(columns={"colDepth":"eco1","lerDepth":"n", "depth":"N"}, inplace=True)
    elif thisfile.lower().startswith("ped"):
        df.rename(columns={"lerDepth":"eco1","colDepth":"n", "depth":"N"}, inplace=True)
    else: 
        if "root" in thisfile:
            df.rename(columns={"lerDepth":"eco1","colDepth":"n", "depth":"N"}, inplace=True)
        else: df.rename(columns={"lerDepth":"n","colDepth":"eco1", "depth":"N"}, inplace=True)
    cols = df.columns.drop("SNP")
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    df.dropna(inplace=True)
    df = df[df["N"]>0]
    filename = thisfile.split(".")[0]
    ## We need a new folder for the output files: check if this exists
    clean_data_path = os.path.join(hetpath, "clean_data/")
    if os.path.exists(clean_data_path) == False:
        os.makedirs(clean_data_path)
    outpath = "clean_data/"
    df.to_csv(clean_data_path + filename + ".csv",index=None)

def format_data(path, dataset):
    print(dataset)
    hetpath = path + "/hetfiles/"
    hompath = path + "/homfiles/"
    ## Load in the raw data files
    hetfiles = [f for f in listdir(hetpath) if isfile(join(hetpath, f))]
    homfiles = [f for f in listdir(hompath) if isfile(join(hompath, f))]
    ## Next line will allow us to ignore .ds_store
    hetfiles = [f for f in hetfiles if not f.startswith('.')]
    homfiles = [f for f in homfiles if not f.startswith('.')]
    ## Get the homograft experiments, rather than the replicates
    homexps = [f.split('_')[0:3] for f in homfiles]
    homexps = ['_'.join(f) for f in homexps]
    homexps = set(homexps)
    ## Choose the correct function
    if dataset=="thieme":
        hom_function = format_hom_data_col_ped
        het_function = format_het_data_col_ped
    #if dataset=="col_ler":
    #    hom_function = format_hom_data_col_ler
    #    het_function = format_het_data_col_ler
    #if dataset=="col_ler_meth":
    #    hom_function = format_hom_data_col_ler_meth
    #    het_function = format_het_data_col_ler_meth
    ## Run the formatting functions
    print("Run hom function")
    [hom_function(hompath, f) for f in homexps]
    print("Run het function")
    [het_function(hetpath, f) for f in hetfiles]

def run_analysis(dataset, hetpath, hompath,filename):
    if dataset == "thieme":
        hom1, hom2 = get_hom_data_thieme(filename)
    if dataset == "col_ler":
        hom1, hom2 = get_hom_data_col_ler(filename)
    if dataset == "col_ler_meth":
        hom1, hom2 = get_hom_data_col_ler_meth(filename)
    hom1 = hompath + hom1
    hom2 = hompath + hom2
    filename = hetpath + filename
    print(hom1, hom2, filename)
    baymob.run_bayes_analysis([hom1, hom2, filename])

path = "raw_data/"
path_thieme = path + "thieme_test"

format_data(path_thieme, "thieme")
## thieme
hetpath = path_thieme + "/hetfiles/clean_data/"
hompath = path_thieme + "/homfiles/clean_data/"
hetfiles = [f for f in listdir(hetpath) if isfile(join(hetpath, f))]
hetfiles = [f for f in hetfiles if not f.startswith('.')]
## Also need to ignore any results files
hetfiles = [f for f in hetfiles if not "results" in f]
hetfiles = [f.replace("txt", "csv") for f in hetfiles]

for f in hetfiles:
    print(f)
    run_analysis("thieme",hetpath, hompath,f)


In [ ]:
## Add in the transcript info for each dataframe
import numpy as np
import pandas as pd

snp_file = "all_snps_annotation.txt"
thieme_path = "raw_data/thieme_test/"
col_ler_path = "raw_data/col_ler/"
col_ler_meth_path = "raw_data/col_ler_meth/"

## Function to count the number of unique transcripts for each SNP
def get_transcript_counts(row):
    ## Get unique values in list
    row_split = row.split(",")
    ## Need to remove splice variants
    row_no_splice = []
    for item in row_split:
        item = item.split("_")[0]
        row_no_splice.append(item)
    row_no_splice = set(row_no_splice)
    return len(row_no_splice)

def create_snp_file(thispath):
    df_snp = pd.read_csv(thispath + snp_file, delimiter = "\t")
    df_snp.dropna(subset =["transcripts"], inplace=True)
    ## Count the number of unique transcripts associated with each SNP
    df_snp["Transcript count"] = df_snp["transcripts"].apply(lambda x: get_transcript_counts(x))
    df_snp["transcripts"] = df_snp["transcripts"].apply(lambda x: x.split('_')[0])
    df_snp = df_snp[df_snp["Transcript count"]==1]
    df_snp[["SNP","transcripts"]].to_csv(thispath + "snp_ref.csv", index=None)

## Apply to all of the data
create_snp_file(thieme_path)
#create_snp_file(col_ler_path)
#create_snp_file(col_ler_meth_path)

# Make large dateset (col_ler / dn_ler)
import os
from os import listdir, mkdir, getcwd
from os.path import isfile, join, isdir
import pandas as pd

col_ler_path = "raw_data/col_ler/"
col_ler_meth_path = "raw_data/col_ler_meth/"
thieme_path = "raw_data/thieme_test/"
snp_file = "raw_data/col_ler_meth/snp_ref.csv"

## Set the snp_ref for each dataset
#col_ler_snp_ref = col_ler_path + snp_file
#col_ler_meth_snp_ref = col_ler_meth_path + snp_file

col_ler_results_path = col_ler_path + "hetfiles/clean_data/"
thieme_results_path = thieme_path + "hetfiles/clean_data/"

## Set the experiments for each dataset
files = [f for f in listdir(thieme_results_path) if isfile(join(thieme_results_path, f))]
results_files = [f for f in files if "results" in f]

## Separate into replicate and experiment
reps = [f for f in files if "results" not in f]

reps = [f.split(".")[0] for f in reps]

exps = [f.split("_")[0:3] for f in results_files]

exps = ["_".join(f) for f in exps]

exps = set(exps)

## Set the results files for each dataset

snp_file = pd.read_csv("raw_data/thieme_test/snp_ref.csv")

thieme_results_path = "raw_data/thieme_test/hetfiles/clean_data/"

files = [f for f in listdir(thieme_results_path) if isfile(join(thieme_results_path, f))]
results_files = [f for f in files if "results" in f]
reps = [f.split(".")[0] for f in files if "results" not in f]

exps = ["Col-FN-root",
         "Col-FN-shoot",
         "Ped-FN-root",
         "Ped-FN-shoot",
         "Col-N-root",
         "Col-P-root",
         "Ped-N-shoot",
         "Ped-P-shoot",
         "FN_stemUpper","FN_flower", "FN_root","FN_stemLower","FN_rosette"]

print(reps)
print(exps)


In [ ]:
## Compare the old and the new results

results_file = files[0]
old_path = "raw_data/thieme/"
new_path = "raw_data/thieme_test/"
file_path = "hetfiles/clean_data/"

df_old = pd.read_csv(old_path + file_path + results_file)
display(df_old)
df_new = pd.read_csv(new_path + file_path + results_file)
display(df_new)

problem_snps = df_old[df_old["log10BF"]==-2]["SNP"].to_list()

print(df_old[df_old["log10BF"]==-2])

df_new[df_new["SNP"].isin(problem_snps)]


In [51]:
## Create one big dataframe

## Load in all of the results files and concatenate into one large dataframe
import numpy as np
from tqdm import tqdm

def create_final_dataframe(exps, reps, results_files, snp_file):
    ## Create the blank dataframe
    df_list = []

    for exp in tqdm(exps):
        these_reps = [x for x in reps if x.startswith(exp)]
        for rep in these_reps:
            ## Load in the results file
            bf_file = pd.read_csv("raw_data/thieme_test/hetfiles/clean_data/" + rep + "_results.csv") 
            bf_file["exp"] = exp
            bf_file["rep"] = rep
            bf_file = pd.merge(bf_file, snp_file, on="SNP")
            df_list.append(bf_file)

    df_final = pd.concat(df_list)
    return df_final

my_df = create_final_dataframe(exps, reps, results_files, snp_file)

100%|██████████| 13/13 [00:06<00:00,  1.99it/s]


In [55]:
my_df.groupby(["transcripts","exp"]).sum().reset_index()[["transcripts","log10BF","exp"]].sort_values("log10BF",ascending=False)

,transcripts,log10BF,exp
105585,AT3G09260,598.064483,FN_stemLower
105584,AT3G09260,307.264481,FN_rosette
7323,AT1G07930,270.483658,FN_root
161584,AT4G21960,252.180413,Ped-FN-shoot
26737,AT1G29930,236.569515,FN_root
...,...,...,...
201422,AT5G24740,-1129.257957,Ped-N-shoot
201423,AT5G24740,-1172.573343,Ped-P-shoot
91124,AT2G42270,-1183.075777,Ped-N-shoot
91123,AT2G42270,-1200.121281,Ped-FN-shoot
